# Interfaces: from protocols to ABCs

## 1. Interfaces and protocols in Python culture

Protocols are defined as the informal interfaces that make polymorphism work in languages with dynamic typing like Python.

Even without an `interface` keyword, every class has an interface.

By definition, protected and private attributes are not part of an interface.

A useful complementary definition of interface is : the subset of an object's public mehtods that enable it to play a speciif role in the system. That's what is implied when the Python documentation mentions `a file-like object` or `an iterable`, without specifying a class.

An interface seen as a set of methods to fulfill a role is what Smalltalkers called a `protocol`, and the term spread to other dynamic language communities.

Protocols are independent of inheritance.

Protocols are interfaces, but because they are informal - defined only by documentation and conventions - protocols cannot be enforced like formal interfaces can.

"X-like object", "X Protocol" and "X interface" are synonyms in the minds of Pythonistas.

## 2. Python digs sequences

One of the most fundamental interfaces in Python is the sequence protocol. The interpreter goes out of its way to handle objects that provide even a minimal implementation of that protocol

`Foo` class only implements one method of the sequence protocol : `__getitem__`

In [1]:
class Foo:
    def __getitem__(self, pos):
        return range(0, 30, 10)[pos]

In [3]:
f = Foo()

In [4]:
f[1]

10

In [5]:
for i in f : print(i)

0
10
20


In [7]:
20 in f

True

In [8]:
15 in f

False

Since Python is smart enough to iterate over `Foo` instances, it can also make the `in` operator work even if `Foo` has no `__contains__` method.

In summary, given the importance of the sequence protocol, in the absence `__iter__` and `__contains__` Python still manages to make iteration and the `in` operartor work by invoking `__getitem__`.

In [9]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA') 
    suits = 'spades diamonds clubs hearts'.split()
    def __init__(self): 
        self._cards = [Card(rank, suit) for suit in self.suits 
                       for rank in self.ranks]
        
    def __len__(self): 
        return len(self._cards)
    
    def __getitem__(self, position): 
        return self._cards[position]

## 3. Monkey-pathching to implement a protocol at run time

`FrenchDeck` acts like a sequence, then it doensn't need its own `shuffle` method, because there is already `random.shuffle` : "Shuffle the sequence x in place"

When you follow established protocols you improve your chances of leverageing existing standard library and third-party code, thanks to duck typing.

In [10]:
from random import shuffle
l = list(range(10))
shuffle(l)

In [11]:
l

[9, 4, 2, 8, 0, 5, 6, 1, 7, 3]

In [13]:
deck = FrenchDeck()
shuffle(deck)

TypeError: 'FrenchDeck' object does not support item assignment

The problem is that shuffle operates by swapping items inside the collection, and `FrechDeck` only implements the `immutable` sequence protocol. Mutalbe sequences must also provide a `__setitem__` method.

Because Python is dynamic, we can fix this at runtime.

In [14]:
def set_card(deck, position, card):
    deck._cards[position] = card

In [15]:
FrenchDeck.__setitem__ = set_card

In [16]:
shuffle(deck)

In [17]:
deck[:5]

[Card(rank='2', suit='spades'),
 Card(rank='5', suit='spades'),
 Card(rank='A', suit='hearts'),
 Card(rank='K', suit='spades'),
 Card(rank='9', suit='hearts')]

`monkey patching` : changing a class or module at run time, withpur touching the source code.

Protocols are dynamic : `random.shuffle` doesn't care what type of argument it gets, it only needs the object to implement part of the mutable sequence protocol.

## 4. Waterfowl and ABCs

duck typing : ignoring an object's actual type, focusing instead on ensuring that the object implements the method names, signatures, and semantics, required for its intended use.

"accidental similarities" happen in programming

```python
class Artist:
    def draw(self):
    
class Gunslinger:
    def draw(self):
    
class Lottery:
    def draw(self):

```

what goose typing means is : `isinstance(obj, cls)` is now just fine ... as long as `cls` is an Abstract Base Class - in other words, `cls`'s metaclass is `abc.ABCMEta`

Python's ABCs add one major paractical advantage : the `register` class method, which lets end-user code "declare: that a certain class becomes a "virtual" subclass of an ABC.

It need not have been developed wiht any awareness of the ABC, and in particular need not inherit from it.

This goes a long way towards breaking the rigidity and strong coupling that make inheritance something to use with much more caution than typically practiced by most OOP programmers.

the use of `isinstance` and `issubclass` becomes more acceptable to test against ABCs.

In the past these functions worked against duck typing, but with ABCs they become more flexible.

On the other hand, it's usually OK to perform an `isinstance` check against an ABC if you must enforce an API contract.

instead of requiring a `list` argument by type checking, simply took the argument and immediately built a `list` from it.

In [18]:
# Duck typing to handle a string or an iterable of strings.
try:
    field_names = field_names.replace(',', ' ').split()
except AttributeError:
    pass

field_names = tuple(field_names)

NameError: name 'field_names' is not defined

## 5. Subclassing an ABC

goose typing in practice.

we'll leverage an existing ABC, `collections.MutalbeSequence`

In [19]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck2(collections.MutableSequence):
    ranks = [str(n) for n in range(2,11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                      for rank in self.ranks]
        
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, position):
        return self._cards[position]
    
    # __setitem__is all we need to enable shuffling.
    def __setitem__(self, position, value):
        self._cards[position] = value
    # But subclassing MutableSequence forces us to implement __delitem__,
    # an abstract method of that ABC.
    def __delitem__(self, position):
        del self._cards[position]
    
    # also required to implement insert, the third abstract method of MutableSequence.
    def insert(self, position, value):
        self._cards.insert(position, value)

Python does not check for the implementation of the abstract methods at import time(when the moduld is loaded and compiled), but only at runtime when we actually try to instantiate `FrenchDeck2`

As a coder of a concrete subclass, you may be able to override methods inherited from ABCs with more efficient implementations.

## 6. ABCs in the standard library

You can find ABCs in the `numbers` and `io` packages.

### ABCs in collections.abc

`Iterable, Container, Sized`:

Every collection should either inherit from these ABCs or at least implement compatible protocols.

`Iterable` supprosts iteration with `__iter__`, `Container` supporst the `in` operator with `__contains__` and `Sized` supports `len()` with `__len__`

`Sequence, Mapping, Set`:

These are the main immutable collection types, each has a mutalbe subclass. 

`MappingView` : 

The objects returned from the mapping methods : `.items()`, `.keys()`, and `.values()` inherit from `ItemView`, `ValuesVeiw` and `ValueVeiw`, respectively.

The first two also inherit the rich interface of `Set`, with all the operators.

`Callable, Hashable`:

Their main use is to support the `isinstance` built-in as a safe way of determining whether an object is callable or hashable.

For callable detection there is the `callable()` built-in function - but there is no equivalent `hashable()` function, so `isinstance(my_obj, Hashable)` is the preferred way to test for a hashable object.

`Iterator`:
Iterator subclasses `Iterable`

### The numbers tower of ABCs

`Number Complex Real Rational Integral`

If you need to check for a integer, use `isinstance(x, numbers.Integral)` to accept `int`, `bool` or other integer types that may be provided by external libraries which register their types with the `numbers` ABCs.

If a value can be a floating point type, you write `isinstance(x, numbers.Real)` 

## 7. Defining and using an ABC

`Tombola` ABC has four methods, The two abstract methods are : 

- load() : put items into the container
- pic() : remove one item at random from the container, returning it.

concrete methods are : 

- loaded() : return True if there is at least one item in the container.
- inspect() : return a sorted tuple built from the items currently in the container, without chaning its contents (its internal ordering is not preserved)

Before ABCs existed, abstract methods would use the statement `raise NotImplementedError` to signal that subclasses were responsible for their implementation.

In [2]:
import abc

# to define an ABC, subclass abc.ABC
class Tombola(abc.ABC):
    
    @abc.abstractmethod
    def load(self, iterable):
        """Add items from an iterable"""
    @abc.abstractmethod
    def pick(self):
        """Remove item at random, returning it.
        
        This method should raise 'LookupError' when the instance is empty.
        """
    #     
    def loaded(self):
        """Return 'True' if there's at least 1 item, 'False' otherwise"""
        return bool(self.inspect())
    
    def inspect(self):
        """Return a sorted tuple with the items currently inside."""
        items = []
        while True:
            try:
                items.append(self.pick())
                
            except LookupError:
                break
                
        self.load(items)
        return tuple(sorted(items))        

An abstract method can actually have an implementation. Even if it does, subclasses will still be forced to override it, but they will be able to invoke the abstract method with `super()`, adding functionality to it instead of implementing from scratch.

The point of this example is to highlight that it's OK to provide concrete methods in ABCs, as long as they only depend on other methods in the interface.

`.inspect()` requires that we catch a `LookupError` thrown by `self.pick()`. The fact that `self.pick()` may raise `LookupError` is also part of its interface, but there is no way to declare this in Python, except in the documentation.

`IndexError` is the `LookupError` subclass raised when we try to get a item from a sequence with an index beyond the last position.

`KeyError` is raised when we use a non-existent key to get an item from a mapping.

In [3]:
class Fake(Tombola):
    def pick(self):
        return 13
    

In [4]:
# The class was created, no errors so far.
Fake

__main__.Fake

In [5]:
f = Fake()

TypeError: Can't instantiate abstract class Fake with abstract methods load

### ABC Syntax details

The best way to declare an ABC is to subclass `abc.ABC` or any other ABC.

If you are using an earlier version of Python, then you must use the `metaclass=` keyword in the `class` statement, pointing to `abc.ABCMeta`

```python
class Tombola(metaclass=abc.ABCMeta):
    #...
```

In Python 2, you must use the `__metaclass__` class attribute:

```python
class Tombola(object): 
    __metaclass__ = abc.ABCMeta
```

A metaclass as a special kind of class, and an ABC is a special kind of class; for example, "regular" classes don't check subclasses, so this is a special behavior of ABCs.

The preferred way to declare an abstract class method is:

```python
class MyABC(abc.ABC):
    @classmethod
    @abc.abstractmethod
    def an_abstract_classmethod(cls, ...):
        pass

```

The order of stacked function decorators usually matters, `abstractmethod()` should be applied as the innermost decorator.

### Subclassing the Tombola ABC


In [6]:
import random

# This BingoCage class explicitly extends Tombola.
class BingoCage(Tombola):
    
    def __init__(self, items):
        # random.SystemRandom implements the random API on top of the os.urandom() function
        # which provides random bytes suitable for cryptographic use.
        self._randomizer = random.SystemRandom()
        self._items = []
        self.load(items)
        
    def load(self, items):
        self._items.extend(items)
        self._randomizer.shuffle(self._items)
        
    def pick(self):
        try:
            return self._items.pop()
        
        except IndexError:
            raise LookupError('pick from empty BingoCage')
            
    def __call__(self):
        self.pick()

`loaded` and `inspect` could be overridden with much faster, one-liners. The point is : we can be lazy and just inherit the sub-optimal concrete methods from an ABC.

In [7]:
import random

class LotteryBlower(Tombola):
    
    def __init__(self, iterable):
        # The initializer accpects any iterable.
        self._balls = list(iterable)
        
    def load(self, iterable):
        self._balls.extend(iterable)
        
    def pick(self):
        try:
            # The random.randrange() function raises ValueError if the range is empty,
            # so we catch that and throw LookupError instead, to be compatible with Tombola.
            position = random.randrange(len(self._balls))
        except ValueError:
            raise LookupError('pick from empty BingoCage')
        # Otherwise the randomly selected item is popped from self._balls.
        return self._balls.pop(position)
    
    def loaded(self):
        return bool(self._balls)
    
    def inspect(self):
        return tuple(sorted(self._balls))
    

In [8]:
a = list([1,2,3,4])

pos = random.randrange(len(a))

In [10]:
pos

2

### A virtual subclass of Tombola

crucial dynamic feature of goose typing : declaring virtual subclasses with the `register` method.

An essential characteristic of goose typing is the ability to register a class as a `virtual subclass` of an ABC, even if it does not inherit from it.

This is done by calling a `register` method on the ABC. The registered class then becomes a virtual subclass of the ABC, and will be recognized as such by functions like `issubclass` and `isinstance`, but it will not inherit any methods or attributes from the ABC.

Virtual subcalsses do not inherit from their registered ABCs, and are not checked for conformance to the ABC interface at any time.

`register` mehtod is usually invoked as a plain function, but it can also be as a decorator.

In [11]:
list.extend

<method 'extend' of 'list' objects>

In [12]:
from random import randrange

# Tombolist is registered as a virtual subclass of Tombola.
@Tombola.register
class TomboList(list):
    
    def pick(self):
        # Tombolist inherits __bool__ from list, and that returns True if the list is not empty.
        if self:
            position = randrange(len(self))
            # pick calls self.pop, inherited from list, passing a random item index.
            return self.pop(position)
        else:
            raise LookupError('pop from empty TomboList')
        # Tombolist.load is the same aas list.extend.   
        load = list.extend
        
    def loaded(self):
        return bool(self)
    
    def inspect(self):
        return tuple(sorted(self))
# standard call syntax
# Tomobla.register(TomboList)

In [14]:
# issubclass(cls, class_or_tuple, /)
# Return whether 'cls' is a derived from another class or is the same class.
issubclass(TomboList, Tombola)

True

In [15]:
t = TomboList(range(100))
# Return whether an object is an instance of a class or of a subclass thereof.
isinstance(t, Tombola)

True

In [16]:
TomboList.__mro__

(__main__.TomboList, list, object)

## 8. How to Tombola subclasses were tested

Two class attributes that allow introspection of a class hierarchy.

`__subclasses__()`

Method that returns a list of immediate subclasses of the class. The list does not include virtual subclasses.

`_abc_registry`

Data attribute - available only in ABCs - that is bound to a `WeakSet` with weak references to registered virtual subclasses of the abstract class.

In [18]:
Tombola.__subclasses__()

[__main__.Fake, __main__.BingoCage, __main__.LotteryBlower]

In [19]:
Tombola._abc_registry

In [20]:
list(Tombola._abc_registry)

[__main__.TomboList]

In [22]:
import doctest

TEST_FILE = 'tombola_tests.rst'
TEST_MSG = '{0:16} {1.attempted:2} tests, {1.failed:2} failed - {2}'

def main(argv):
    verbose = '-v' in argv
    # __subclasses__() : lists the direct descendants that are alive in memory.
    real_subclasses = Tombola.__subclasses__()
    
    # Build a list from _abc_registry(which is a WeakSet) so we can concatenate
    # it with the result of __subclasses__()
    virtual_subclasses = list(Tombola._abc_registry)
    
    for cls in real_subclasses +  virtual_subclasses:
        test(cls, verbose)
        
def test(cls, verbose=False):
    res = doctest.testfile(TEST_FILE,
                          globs={'ConcreteTombola': cls},
                          verbose=verbose,
                          optionflags=doctest.REPORT_ONLY_FIRST_FAILURE)
    
    tag = 'FAIL' if res.failed else 'OK'
    print(TEST_MSG.format(cls.__name__, res, tag))
    
    
if __name__ == '__main__':
    import sys
    main(sys.argv)

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\Program Files\\Anaconda3\\lib\\site-packages\\ipykernel\\tombola_tests.rst'

In [ ]:
balls = list(range(3))
globe = Co

## 9. Usage of register in practice

Even if `register` can now be used as decorator, it's more widely deployed as a function to register classes defined elesewhere.

Several other built-in types are registered to ABCs in `_collection_abc.py`
Those registerations happen only when that module is imported.

## 10. Gees can behave as ducks

A class can be recognized as a virtual subclass of an ABC even without registeration.

In [23]:
class Struggle:
    def __len__(self) :return 23
    

In [24]:
from collections import abc

isinstance(Struggle(), abc.Sized)

True

In [26]:
issubclass(Struggle, abc.Sized)

True

Because `abc.Sized` implements a special class method anmed `__subclassbook__`

In [ ]:
class Sized(metaclass=ABCMeta):
    
    __slots__ = ()
    
    @abstractmehtod
    def __len__(self):
        return 0
    
    @classmethod
    def __subclasshook__(cls, C):
        if cls is Sized:
            # If there is an attribute named __len__ in the __dict__ of any class listed
            # in C.__mro__ (i.e C and its superclasses)
            if any("__len__" in B.__dict__ for B in C.__mro__):
                # return True, signalling that C is a virtual subclass of Sized.
                return True
            
        return NotImplemented

You can have formal interface definitions with ABCs, you can make `isinstance` checks everywhere, and still have a completely unrelated class play along just because it implements a certain method.

This only works for ABCs that do provide a `__subclasshook__`

## Summary

- highly dynamic nature of informal interfaces(protocls)
- static interface declarations of ABCs
- dynamic side of ABCs : virtual subclasses and dynamic subclass detection with `__subclasshook__`

Interface were informal like the protocols from Smalltalke, and the official docs used language such as "foo protocol", "foo interface" and "foo-like object" interchangeably.

Protocol-style interfaces have nothing to do with inheritance; each class stands alone when implementing a protocol. That's how interfaces look like when you embrace duck typing.

We then went back to the old `FrenchDeck` example to support shuffling by dynamically adding a method. This illustrated monkey patching and emphasized the dynamic nature of protocols.

gooes typing to describe a new style of Pythong programming.

With "goose typing", ABCs are used to make interfaces explicit and classes may claim to implement an interface by subclassing an ABC or by registering with it

Main drawbacks and advantages of explicit ABCs. 

Inheriting from `abc.MutableSequence` forced us to implement two methods we did not really need.


Several built-in types are registered to ABCs in the `collections.abc` module so you can ask `isinstance(memoryview, abc.Sequence)` and get `True`, even if `memoryview` does not inherit from `abc.Sequence`.

`__subclasshook__` magic which lets an ABC recognize any unregistered class as a subclass.